查看使用SRNet-Modular得出的表达式

In [1]:
import os
import torch
import sympy as sp
from utils import load_pickle
from load_data import load_sr_data
from SRNet import srnets
from neural_network import neural_networks

def load_srnet(path_dir, net, args):
    sr_param = torch.load(os.path.join(path_dir, "sr_param"))
    _, srnet_class = srnets[args.srnet_name]
    srnet = srnet_class(sr_param=sr_param, neural_network=net)
    srnet.load_state_dict(torch.load(os.path.join(path_dir, "srnet")))
    srnet.assign_genes(load_pickle(os.path.join(path_dir, "srnet_genes")))
    return srnet

event_dir = "./output/compared"
net_dir = "./output/MLPs"
datasets = ["kkk0","feynman0","529_pollen"]

for dataset in datasets:
    srnet_dir = os.path.join(event_dir, f"mlp-{dataset}")
    net_path = os.path.join(net_dir, f"MLP-{dataset}", "mlp", "mlp")
    args = torch.load(os.path.join(srnet_dir, "args"))
    train_dataset, _ = load_sr_data(args.dataset)
    n_inputs, n_outputs = train_dataset.X.size(-1), 1

    # load net
    nn_class = neural_networks["MLP"]
    net = nn_class(n_inputs, n_outputs)
    net.load_state_dict(torch.load(net_path))
    net.eval()

    # load sr
    srnet = load_srnet(srnet_dir, net, args)

    exps = [layer.expr(mul_w=False)[0] for layer in srnet.explained_layers]
    for i, expr in enumerate(exps):
        expr = sp.simplify(expr).evalf(4)
        exps[i] = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
    hidx = 1
    for i, expr in enumerate(exps[1:]):
        for xidx in range(16):
            if f"x{xidx}" in expr:
                exps[i+1] = exps[i+1].replace(f"x{xidx}", "(hat{h}^"+f"{hidx})_{xidx}")
        hidx += 1
    print(f'Dataset {dataset} Each Layer Expression:{exps}')
    final_expr = srnet.expr()[0]
    final_expr = sp.simplify(final_expr).evalf(4)
    final_expr = str(final_expr.xreplace({n : round(n, 2) for n in final_expr.atoms(sp.Number)})).replace("**", "^")
    print(f'Dataset {dataset} Final Expression:{final_expr}')

查看vKANPDE得到的关于图像的偏微分方程表达式

In [25]:
import os
import torch
from SRNet.vkan_models import vKANPDE
from SRNet.parameters import vKANParameter
class Args:
    n_layer=5
    function_set=None
    width=[3,1,1]
    grid=5
    k=3
    noise_scale=0.1
    noise_scale_base=0.1
    base_fun=torch.nn.SiLU()
    symbolic_enabled=True
    bias_trainable=True
    grid_eps=0.02
    grid_range=[-1, 1]
    sp_trainable=True
    sb_trainable=True
    device='cpu'
arg=Args()
vKANPDE_path = "output/find-pde_with_vkan/test/vKANPDE.pt"
param = vKANParameter(
    n_inputs=5,  # x, y, t, dx?, dy?
    n_outputs=1,  # u
    n_eph=0,
    args=arg,
    function_set=None,
    one_in_one_out=False
)
PDE = vKANPDE(param, with_fu=False)
PDE.load_state_dict(torch.load(vKANPDE_path, map_location=torch.device('cpu')))
PDE.expr()

查看EQLPDE得到的关于图像的偏微分方程表达式（原版论文）

In [17]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr5e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()

expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr)  # 根据output/find_pde路径下的PDE.pt，得到图像的偏微分方程

In [10]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr5e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()

expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr)  # 根据output/find_pde路径下的PDE.pt，得到图像的偏微分方程

查看EQLPDE得到的关于图像的偏微分方程表达式（改进版）

In [11]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr5e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()

expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr)  # 根据output/find_pde路径下的PDE.pt，得到图像的偏微分方程

In [12]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr5e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()

expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr)  # 根据output/find_pde路径下的PDE.pt，得到图像的偏微分方程

# 原版

第一层卷积：-0.04*dx - 0.01*dy - 0.18*log(Abs(0.16*dx + 0.03*dy - 0.75)) + 0.01*sin(0.51*dx + 0.03*dy) + 0.14*cos(0.16*dy - 0.19) - 0.19

第二层卷积：-0.04*dx - 0.01*dy - 0.18*log(Abs(0.16*dx + 0.03*dy - 0.75)) + 0.01*sin(0.33*dx - 0.01) + 0.14*cos(0.01*dx - 0.15*dy + 0.16) - 0.19

# 改进版

第一层卷积：-0.13*log(Abs(0.01*dx + 0.02*dy - 0.78)) - 0.03

第二层卷积：-0.14*log(Abs(0.02*dx + 0.02*dy - 0.77)) + 0.15*cos(0.02*dx + 0.04*dy - 0.12) - 0.18

查看EQLPDE得到的关于图像的偏微分方程表达式（原版论文）

In [13]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr3e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()

expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr)  # 根据output/find_pde路径下的PDE.pt，得到图像的偏微分方程

In [14]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr3e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()

expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr)  # 根据output/find_pde路径下的PDE.pt，得到图像的偏微分方程

查看EQLPDE得到的关于图像的偏微分方程表达式（改进版）

In [10]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr3e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()

expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr)  # 根据output/find_pde路径下的PDE.pt，得到图像的偏微分方程

In [9]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr3e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()

expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr)  # 根据output/find_pde路径下的PDE.pt，得到图像的偏微分方程

# 原版

第一层卷积：-0.e-2*dx^2 - 0.08*dx - 0.01*dy - 0.22*log(Abs(0.19*dx + 0.01*dy - 0.69)) + 0.04*sin(0.63*dx + 0.03*dy - 0.03) + 0.16*cos(0.18*dy - 0.18) - 0.23

第二层卷积：-0.e-2*dx^2 - 0.07*dx - 0.01*dy - 0.22*log(Abs(0.16*dx + 0.03*dy - 0.7)) + 0.03*sin(0.5*dx + 0.05*dy - 0.04) + 0.15*cos(0.15*dy - 0.17) - 0.23

# 改进版

第一层卷积：-0.e-2*dx - 0.e-2*dy - 0.2*log(Abs(0.02*dx + 0.02*dy - 0.71)) + 0.16*cos(0.02*dx + 0.03*dy - 0.15) - 0.22

第二层卷积：-0.e-2*dx - 0.19*log(Abs(0.03*dx + 0.01*dy - 0.72)) + 0.15*cos(0.02*dx + 0.05*dy - 0.16) - 0.21

将dxdy换成dt后的结果

改进版

In [8]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde_dt/5layers-li0-clip0-lr5e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 5e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde_dt/5layers-li0-clip0-lr3e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 3e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde_dt/5layers-li1-clip0-lr5e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 5e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde_dt/5layers-li1-clip0-lr3e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 3e-3: {expr[10:-3]}")


In [12]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde_dt/5layers-li0-clip0-lr1e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 1e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde_dt/5layers-li1-clip0-lr1e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 1e-3: {expr[10:-3]}")


原版

In [11]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde_dt/5layers-li0-clip0-lr5e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 5e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde_dt/5layers-li0-clip0-lr3e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 3e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde_dt/5layers-li1-clip0-lr5e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 5e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde_dt/5layers-li1-clip0-lr3e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dt'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 3e-3: {expr[10:-3]}")


# 原版：

🌟 layer 1, lr 5e-3: 0.02*dt^2 + 0.04*dt*dx - 0.e-2*dt - 0.02*dx^2 - 0.02*dx - 0.16*log(Abs(0.02*dt + 0.11*dx - 0.76)) + 0.14*cos(0.06*dt - 0.18) - 0.19

layer 1, lr 3e-3: 0.03*dt^2 + 0.06*dt*dx - 0.e-2*dt - 0.03*dx^2 - 0.03*dx + 0.01*dy - 0.2*log(Abs(0.02*dt + 0.08*dx - 0.02*dy - 0.71)) - 0.e-2*sin(0.4*dt - 0.54*dx) + 0.15*cos(-0.09*dt + 0.03*dx + 0.07*dy + 0.17) - 0.22

layer 2, lr 5e-3: 0.e-2*dt^2 + 0.01*dt*dx - 0.e-2*dt - 0.e-2*dx^2 - 0.02*dx - 0.17*log(Abs(0.04*dt + 0.07*dx + 0.02*dy - 0.74)) + 0.14*cos(0.07*dt - 0.16) - 0.19

🌟 layer 2, lr 3e-3: 0.e-2*dt^2 + 0.01*dt*dx - 0.02*dt - 0.e-2*dx^2 - 0.02*dx - 0.e-2*dy - 0.21*log(Abs(0.07*dt + 0.05*dx + 0.02*dy - 0.7)) - 0.e-2*sin(0.39*dt - 0.43*dx - 0.03*dy + 0.02) + 0.15*cos(-0.07*dt + 0.02*dx + 0.17) - 0.22

# 改进版：

🌟 layer 1, lr 5e-3: 0.13*log(Abs(0.02*dt + 0.01*dx + 0.02*dy - 0.75)) + 0.15*cos(0.01*dx - 0.08) - 0.19

layer 1, lr 3e-3: 0.19*log(Abs(0.01*dx + 0.01*dy - 0.7)) + 0.16*cos(0.01*dt + 0.02*dx + 0.02*dy - 0.14) - 0.22

layer 2, lr 5e-3: 0.e-2*dx - 0.18*log(Abs(0.02*dx - 0.72)) + 0.15*cos(0.03*dt + 0.02*dy - 0.14) - 0.21

🌟 layer 2, lr 3e-3: 0.e-2*dx - 0.e-2*(0.12*dx + 0.04*dy - 1)^2 - 0.2*log(Abs(0.02*dx - 0.71)) - 0.06

原版

In [14]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr5e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 5e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr3e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 3e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr5e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 5e-3: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr3e-3-pw1.0-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 3e-3: {expr[10:-3]}")

改进版

In [6]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr5e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 5e-3: {expr[10:-3]}")

# 换个实验结果看看表达式，不使用EQL的稀疏回归
pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr5e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 5e-3, without sparse_filter: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr3e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 3e-3: {expr[10:-3]}")

# 换个实验结果看看表达式，不使用EQL的稀疏回归
pinn_eql = torch.load("output/find-pde/5layers-li0-clip0-lr3e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 1, lr 3e-3, without sparse_filter: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr5e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 5e-3: {expr[10:-3]}")

# 换个实验结果看看表达式，不使用EQL的稀疏回归
pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr5e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 5e-3, without sparse_filter: {expr[10:-3]}")

# 换个实验结果看看表达式
pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr3e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.01)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 3e-3: {expr[10:-3]}")

# 换个实验结果看看表达式，不使用EQL的稀疏回归
pinn_eql = torch.load("output/find-pde/5layers-li1-clip0-lr3e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(f"layer 2, lr 3e-3, without sparse_filter: {expr[10:-3]}")

# 原版：

🌟 layer 1, lr 5e-3: 0.04*dx - 0.01*dy - 0.18*log(Abs(0.16*dx + 0.03*dy - 0.75)) + 0.01*sin(0.51*dx + 0.03*dy) + 0.14*cos(0.16*dy - 0.19) - 0.19

layer 1, lr 3e-3: 0.e-2*dx^2 - 0.08*dx - 0.01*dy - 0.22*log(Abs(0.19*dx + 0.01*dy - 0.69)) + 0.04*sin(0.63*dx + 0.03*dy - 0.03) + 0.16*cos(0.18*dy - 0.18) - 0.23

layer 2, lr 5e-3: 0.04*dx - 0.01*dy - 0.18*log(Abs(0.16*dx + 0.03*dy - 0.75)) + 0.01*sin(0.33*dx - 0.01) + 0.14*cos(0.01*dx - 0.15*dy + 0.16) - 0.19

🌟 layer 2, lr 3e-3: 0.e-2*dx^2 - 0.07*dx - 0.01*dy - 0.22*log(Abs(0.16*dx + 0.03*dy - 0.7)) + 0.03*sin(0.5*dx + 0.05*dy - 0.04) + 0.15*cos(0.15*dy - 0.17) - 0.23

# 改进版：

🌟 layer 1, lr 5e-3: 0.13*log(Abs(0.01*dx + 0.02*dy - 0.78)) - 0.03

layer 1, lr 3e-3: 0.e-2*dx - 0.e-2*dy - 0.2*log(Abs(0.02*dx + 0.02*dy - 0.71)) + 0.16*cos(0.02*dx + 0.03*dy - 0.15) - 0.22

layer 2, lr 5e-3: 0.14*log(Abs(0.02*dx + 0.02*dy - 0.77)) + 0.15*cos(0.02*dx + 0.04*dy - 0.12) - 0.18

🌟 layer 2, lr 3e-3: 0.e-2*dx - 0.19*log(Abs(0.03*dx + 0.01*dy - 0.72)) + 0.15*cos(0.02*dx + 0.05*dy - 0.16) - 0.21

In [6]:
import torch
import sympy as sp
from SRNet.usr_models import EQLParameter, EQLPDE

device=torch.device("cpu")

class Args:
    n_layers = 5
    def __init__(self, n_layers=5) -> None:
        self.n_layers = n_layers

args = Args()
param = EQLParameter(
    n_inputs=5, # x, y, t, dx?, dy?
    n_outputs=1, # u
    n_eph=0,
    args=args,
    function_set=["add", "mul", "sin", "cos", "square", "log"]
)

pinn_eql = torch.load("output/find-pde_rnn/5layers-li0-clip0-lr1e-3-pw1e-5-pew1.0-wd1e-5-w0.1/PDE.pt", map_location=device)
PDE = EQLPDE(param, with_fu=False)
PDE.load_state_dict(pinn_eql)
PDE.eval()
expr = PDE.expr(input_vars=['dx', 'dy', 'dxdy'], sparse_filter=0.1)
expr = sp.simplify(expr)
expr = str(expr.xreplace({n : round(n, 2) for n in expr.atoms(sp.Number)})).replace("**", "^")
print(expr[10:-3])